# Content Based Recommender System

This is a simple approach of recommending items based on the attributes of the item itself. For example, 
if we know that someone enjoys watching movies of a particular genre, we would try to recommned the person 
other movies of the same genre. 

In this case we will look at the product description of the items that a user has already purchased, and
recommend items that are similar to the item already purchased.

One of the most prominent advantage of this system is that, it does not require a big user base to recommend items. This way it avoids the 'cold start' problem. Since content can be present in varied form, they open up the options to use different approaches like: text processing techniques, the use of semantic information etc.


## Import required libraries

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 100

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("pastel")
sns.set_style("white")
sns.set_context("paper", font_scale=1.35)
%matplotlib inline

#Importing TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

import warnings
warnings.filterwarnings('ignore')

## Read the data file into a pandas dataframe

In [2]:
lb = pd.read_csv('luxury_beauty_cleaned.csv')
lb.head()

,rating,userId,itemId,userName,description,title,price,rating_count,rating_avg
0,2.0,A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ,30.0,582,4.623711
1,5.0,A3HO2SQDCZIE9S,B00004U9V2,chandra,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ,30.0,582,4.623711
2,5.0,A2EM03F99X3RJZ,B00004U9V2,Maureen G,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ,30.0,582,4.623711
3,5.0,A3Z74TDRGD0HU,B00004U9V2,Terry K,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ,30.0,582,4.623711
4,5.0,A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ,30.0,582,4.623711


In [3]:
print("No of unique users in the data : ",lb['userId'].nunique())
print("No of unique products in the data : ",lb['itemId'].nunique())

No of unique users in the data :  342713
No of unique products in the data :  6854


In [4]:
# we will create a dataframe of unique products, their descriptions and title
# this dataframe will be used to build the content based recommender

lb_products = lb[['itemId', 'description', 'title']]
lb_products.drop_duplicates(subset=['itemId'], inplace=True)
lb_products.drop_duplicates(subset=['title'], inplace=True)
lb_products.reset_index(inplace = True)

lb_products.drop(columns=['index'], inplace=True)

lb_products.head()

,itemId,description,title
0,B00004U9V2,"['After a long day of handling thorny situations, our new hand therapy pump is just the help you...",Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ
1,B00005A77F,['This liquid soap with convenient pump dispenser is formulated with conditioning extracts of sa...,"Crabtree & Evelyn Hand Soap, Gardeners, 10.1 fl. oz."
2,B00005NDTD,"[""Remember why you love your favorite blanket? The soft, comforting feeling of wrapping it aroun...",Soy Milk Hand Crme
3,B00005V50C,"['<P><STRONG>Please note: Due to product improvements, the image shown may temporarily appear di...",Supersmile Powdered Mouthrinse
4,B00005V50B,"['Created by Dr. Irwin Smigel, world-renowned ""Father of Aesthetic Dentistry"". Contains the unpr...","Supersmile Professional Teeth Whitening Toothpaste Recommended By Cosmetic Dentists, CLINICALLY..."


# Build content based product recommender using product descriptions

In [5]:
#Defining a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

#Replacing NaN with an empty string
lb['description'] = lb_products['description'].fillna('')

In [6]:
#Constructing the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(lb_products['description'])

In [7]:
tfidf_matrix.shape

# There are 78928 unique words in the product descriptions

(6575, 78926)

In [8]:
# Compute the sigmoid kernel
lin = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
lin[0]

array([1.        , 0.65599221, 0.02758466, ..., 0.0028182 , 0.00474063,
       0.0030886 ])

In [10]:
# Reverse mapping of indices and product titles
indices = pd.Series(lb_products.index, index=lb_products['title']).drop_duplicates()

In [11]:
indices.head(20)

title
Crabtree & Evelyn - Gardener's Ultra-Moisturising Hand Therapy Pump - 250g/8.8 OZ                       0
Crabtree & Evelyn Hand Soap, Gardeners, 10.1 fl. oz.                                                    1
Soy Milk Hand Crme                                                                                      2
Supersmile Powdered Mouthrinse                                                                          3
Supersmile Professional Teeth Whitening Toothpaste  Recommended By Cosmetic Dentists, CLINICALLY...     4
Archipelago Morning Mint Body Lotion ,18 Fl Oz                                                          5
Calvin Klein ck one Eau de Toilette, 3.4 fl. oz.                                                        6
Helen of Troy 1193 Professional Flat Iron                                                               7
DERMAdoctor Calm, Cool & Corrected anti-redness tranquility cream - 1.7 Oz                              8
HOT TOOLS Professional 24K Gold Marcel I

In [12]:
# This function will give a list of top 10 similar products base on product description
# The function takes in product title as input parameter

def give_rec(title, lin=lin):
    
    # Get the index corresponding to title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    lin_scores = list(enumerate(lin[idx]))

    # Sort the products 
    lin_scores = sorted(lin_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar products
    lin_scores = lin_scores[1:11]

    # Movie indices
    product_indices = [i[0] for i in lin_scores]

    # Top 10 most similar products
    return lb_products['title'].iloc[product_indices].to_list()

In [13]:
def getListofRatedItems(userId):
    a = lb[lb['userId']==userId][['title']].to_dict('list')
    return a['title']

In [14]:
def getRecforUser(userId):
    print("Getting recommendations for user {}....\n".format(userId))
    
    # get the list of itemids rated by the user
    item_list = getListofRatedItems(userId)
    
    # call give_rec function for each product in item_list
    for item in item_list:
        print("Products similar to '{}' are ....\n".format(item))
        for sim_item in give_rec(item):
            print(sim_item)
        print('\n')

## Evaluating the recommender system

In [15]:
getRecforUser('A15KL7ZPB6SJ16')

Getting recommendations for user A15KL7ZPB6SJ16....

Products similar to 'Helen of Troy 1193 Professional Flat Iron' are ....

Hot Tools Professional Nano Ceramic Flipperless Curling Wand for Shiny Curls, 1 ¼ Inch
Hot Tools Professional Ceramic + Tourmaline Marcel Iron/Wand, ¾ Inches
Hot Tools Micro Crimper, 1 Inch
HOT TOOLS Spring Curling Iron, Black/White
Hot Tools Professional Nano Ceramic Bubble Curling Wand for Beachy Waves, 1¼ Inches
HOT TOOLS PINKTITANIUM Salon Titanium Curling Iron
HOT TOOLS HOT TOOLS PINKTITANIUM Salon Titanium Curling Iron
Hot Tools Professional Ceramic + Tourmaline Curling Iron/Wand for Healthy Looking Curls, 2 Inches
Hot Tools HTG1860 Flipperless Curling Wand, Gold/Black
HOT TOOLS Professional Silicone Bristle Hot Brush Styler for Added Volume, 1 Inch




In [17]:
getRecforUser('A1SLHI58I09T1K')

Getting recommendations for user A1SLHI58I09T1K....

Products similar to 'Dermablend Cover Creme Full Coverage Foundation with SPF 30, 1 Oz' are ....

Dermablend Quick-Fix Full Coverage Concealer, 0.16 Oz.
Dermablend Quick-Fix Body Makeup Full Coverage Foundation Stick, 0.42 Oz.
Dermablend Smooth Liquid Concealer
Dermablend Smooth Liquid Foundation with SPF 25, 1 Fl. Oz.
Dermablend Intense Powder High Coverage Foundation
Dermablend Loose Setting Powder
Dermablend Makeup Setting Spray Set + Refresh for Long Lasting Makeup Wear, 3.4 Fl. Oz.
37 Actives High Performance Anti-Aging Treatment Foundation, 1.0 Fl Oz
Glo Skin Beauty Oil Free Camouflage
La Roche-Posay Effaclar BB Blur with SPF 20, 1.01 Fl. Oz.


Products similar to 'Vichy Pureté Thermale One Step Micellar Cleansing Water' are ....

Vichy Homme Hydra Mag C Body Wash & Hair Shower Gel, 6.76 Fl. Oz.
Vichy Dermafinish Corrective Full Coverage Concealer Stick, 14 Hour Color Wear, 0.16 Oz.
Vichy Mineralizing Thermal Water Rich in 15 M